In [16]:
# pour les imports
import numpy as np


# Zayed

In [17]:
class Cards:
    def __init__(self):
        self.ncards = 20  # number of cards
        self.cards = 4*[7, 8, 9, 10, 11]  # cards (?? 11 pour "A")

    def shuffle(self):
        '''Knuth shuffle'''
        np.random.shuffle(self.cards)

    def hands(self):
        '''distribution of cards by the agent :)'''
        self.hand1 = [self.cards[0], self.cards[2]]
        self.hand2 = [self.cards[1], self.cards[3]]
        # self.sumHand1 = self.cards[0]+self.cards[2]  # the sum of the two cards
        # # for agent (agent is the one who shuffles)
        # self.sumHand2 = self.cards[1]+self.cards[3]


In [18]:
class Enemies :
    '''many enemies'''
    def __init__(self):
        self.maxPoints= 10000  # the maximum amount of points that the enemy has
        self.points= [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

    def enemy1(self, hand1): # the enemy hand
        goforpoints = None
        if hand1[0]+hand1[1] >= 18:  # (the mean of all possible hands)
            goforpoints = np.random.choice(self.points[5:])
        else:
            goforpoints = np.random.choice(self.points[:5])
        return goforpoints



In [19]:
# class Greedy :
#     def __init__(self):
#         self.maxPoints= 10000  # the maximum amount of points that the greedy agent has
#         self.points= [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    
#     def 
    

In [33]:
def mapTo1422(sampled,a14,b22):

    return int((b22-a14)*sampled+a14) # the distribution of an arm (a point) ??

In [35]:
class ThompsonSampling :
    def __init__(self,alist,blist):
        self.maxPoints= 10000  # the maximum amount of points that the agent has
        self.points = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
        # we can keep track of this beta =>  the agent learn  ?????
        self.alist = alist
        self.blist = blist

    def chooseArmToGo(self,hand2,enemyResponse): # like an arm
        # sample the distrubution of the arm number enemyResponse
        estimated =np.random.beta(self.alist[enemyResponse-1],self.blist[enemyResponse-1])

        estimatedSumEnemyHand=mapTo1422(estimated,14,22) # according to the agent technique
        if(np.sum(hand2)>=estimatedSumEnemyHand): # if the agent hand is grater the the sampled enamy hand
            return True # go for
        else :
            return False # go for -1
    def updateBeta(self, gowith,enemyResponse,hand1,hand2):
        h2=np.sum(hand2) # agent
        h1=np.sum(hand1)
        if(gowith) and h1>h2 :
            self.alist[enemyResponse-1]+=1 # shift bata to the right
        if  h1<=h2 and not (gowith):
            self.blist[enemyResponse-1]+=1 # shift bata to the left




    

In [36]:
# test
enemy=Enemies()
a=b=[1 for i in range(10)]
agent=ThompsonSampling(a,b)
while(enemy.maxPoints>=10 and agent.maxPoints>=10):
    cards = Cards()
    cards.shuffle()
    cards.hands()
    hand1= cards.hand1
    hand2=cards.hand2
    EnemyGoFor=enemy.enemy1(hand1)
    AgentGo=agent.chooseArmToGo(hand2,EnemyGoFor)
    #show cards
    if(AgentGo):
        if(np.sum(hand1)>np.sum(hand2)):
            # agent loses
            agent.maxPoints-=EnemyGoFor
        if(np.sum(hand2)>np.sum(hand1)):
            # agent loses
            enemy.maxPoints-=EnemyGoFor
    else :
        agent.maxPoints-=1
    # update agent strategy
    agent.updateBeta(AgentGo,EnemyGoFor,hand1,hand2)
    print("=========Enemy========")
    print("hand1 ", hand1)
    print("GoFor", EnemyGoFor)
    print("maxPonts= ", enemy.maxPoints)
    print("=========Agent========")
    print("hand2 ", hand2)
    print("GoFor",AgentGo)
    print("maxPonts= ", agent.maxPoints)
if(enemy.maxPoints <10):
    print("Enemy losed")
if(agent.maxPoints <10):
    print("Agent losed")



=========Enemy========
hand1  [10, 11]
GoFor 7
maxPonts=  10000
=========Agent========
hand2  [7, 11]
GoFor True
maxPonts=  9993
=========Enemy========
hand1  [10, 8]
GoFor 7
maxPonts=  9993
=========Agent========
hand2  [11, 10]
GoFor True
maxPonts=  9993
=========Enemy========
hand1  [8, 10]
GoFor 6
maxPonts=  9987
=========Agent========
hand2  [11, 9]
GoFor True
maxPonts=  9993
=========Enemy========
hand1  [9, 8]
GoFor 5
maxPonts=  9982
=========Agent========
hand2  [7, 11]
GoFor True
maxPonts=  9993
=========Enemy========
hand1  [10, 11]
GoFor 10
maxPonts=  9982
=========Agent========
hand2  [7, 10]
GoFor False
maxPonts=  9992
=========Enemy========
hand1  [11, 9]
GoFor 6
maxPonts=  9982
=========Agent========
hand2  [10, 8]
GoFor True
maxPonts=  9986
=========Enemy========
hand1  [10, 9]
GoFor 7
maxPonts=  9982
=========Agent========
hand2  [10, 7]
GoFor True
maxPonts=  9979
=========Enemy========
hand1  [9, 11]
GoFor 8
maxPonts=  9982
=========Agent========
hand2  [11, 8]
GoFor 

# Abdallahi

# Mermoud